In [42]:
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from sqlalchemy.orm import sessionmaker
import os

SOURCE_DB_NAME = 'source.db'
SOURCE_DB_CONNECTION = 'sqlite:///{name}'.format(name=SOURCE_DB_NAME)
TARGET_DB_NAME = 'target.db'
TARGET_DB_CONNECTION = 'sqlite:///{name}'.format(name=SOURCE_DB_NAME)

In [43]:
if os.path.exists(SOURCE_DB_NAME):
  os.remove(SOURCE_DB_NAME)

source_engine = create_engine(SOURCE_DB_CONNECTION, echo = True)
SourceSession = sessionmaker()
SourceSession.configure(bind=source_engine)
source_session = SourceSession()

t_projects = text(
    """
    CREATE TABLE projects(
        id INT PRIMARY KEY, 
        name VARCHAR NOT NULL
    )
    """)

t_users = text(
    """
    CREATE TABLE users(
        id INT PRIMARY KEY, 
        name VARCHAR NOT NULL, 
        project_id INT REFERENCES projects(id) NOT NULL
    )
    """)

i_projects = text("""INSERT INTO projects(id, name) VALUES(:id, :name)""")
projects = (
    { "id": 1, "name": "A" },
    { "id": 2, "name": "B" },
)

i_users = text("""INSERT INTO users(id, name, project_id) VALUES(:id, :name, :project_id)""")
users = (
    { "id": 1, "name": "Shivam", "project_id": 1 },
    { "id": 2, "name": "Kishor", "project_id": 2 },
)

source_session.execute(t_projects)
source_session.execute(t_users)

for user in users:
    source_session.execute(i_users, user)

for project in projects:
    source_session.execute(i_projects, project)

source_session.commit()  

2021-06-04 16:57:42,715 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-04 16:57:42,718 INFO sqlalchemy.engine.Engine 
    CREATE TABLE projects(
        id INT PRIMARY KEY, 
        name VARCHAR NOT NULL
    )
    
2021-06-04 16:57:42,720 INFO sqlalchemy.engine.Engine [generated in 0.00221s] ()
2021-06-04 16:57:42,929 INFO sqlalchemy.engine.Engine 
    CREATE TABLE users(
        id INT PRIMARY KEY, 
        name VARCHAR NOT NULL, 
        project_id INT REFERENCES projects(id) NOT NULL
    )
    
2021-06-04 16:57:42,931 INFO sqlalchemy.engine.Engine [generated in 0.00125s] ()
2021-06-04 16:57:43,052 INFO sqlalchemy.engine.Engine INSERT INTO users(id, name, project_id) VALUES(?, ?, ?)
2021-06-04 16:57:43,053 INFO sqlalchemy.engine.Engine [generated in 0.00106s] (1, 'Shivam', 1)
2021-06-04 16:57:43,056 INFO sqlalchemy.engine.Engine INSERT INTO users(id, name, project_id) VALUES(?, ?, ?)
2021-06-04 16:57:43,059 INFO sqlalchemy.engine.Engine [cached since 0.006355s ago] (2, 'Kisho

In [44]:
v_dw = text(
    """
    CREATE VIEW v_dw 
    AS 
    SELECT 
        users.id AS user_id,
        users.name AS user_name,
        projects.id AS project_id,
        projects.name AS project_name
    FROM 
        users, projects 
    WHERE 
        users.project_id = projects.id
    """)

source_session.execute(v_dw)
source_session.commit()

2021-06-04 16:57:43,360 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-04 16:57:43,362 INFO sqlalchemy.engine.Engine 
    CREATE VIEW v_dw 
    AS 
    SELECT 
        users.id AS user_id,
        users.name AS user_name,
        projects.id AS project_id,
        projects.name AS project_name
    FROM 
        users, projects 
    WHERE 
        users.project_id = projects.id
    
2021-06-04 16:57:43,363 INFO sqlalchemy.engine.Engine [generated in 0.00152s] ()
2021-06-04 16:57:43,550 INFO sqlalchemy.engine.Engine COMMIT


In [45]:
!sqlacodegen --tables v_dw --outfile models.py {SOURCE_DB_CONNECTION}

In [46]:
target_engine = create_engine(TARGET_DB_CONNECTION, echo = True)
TargetSession = sessionmaker()
TargetSession.configure(bind=target_engine)
target_session = TargetSession()

In [47]:
from models import metadata

metadata.create_all(target_engine)

2021-06-04 16:57:50,713 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-04 16:57:50,715 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("v_dw")
2021-06-04 16:57:50,717 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-06-04 16:57:50,721 INFO sqlalchemy.engine.Engine COMMIT


In [48]:
def filter_row(row):
    if row['user_id'] == 2:
        return row, True
    else:
        return row, False

In [49]:
from models import t_v_dw

batch = []

for row in source_session.query(t_v_dw).all():
    data, is_valid = filter_row(dict(row))
    if is_valid:
        batch.append(data)

for data in batch:
    target_session.execute(t_v_dw.insert(values=data))

2021-06-04 16:57:52,334 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-04 16:57:52,340 INFO sqlalchemy.engine.Engine SELECT v_dw.user_id AS v_dw_user_id, v_dw.user_name AS v_dw_user_name, v_dw.project_id AS v_dw_project_id, v_dw.project_name AS v_dw_project_name 
FROM v_dw
2021-06-04 16:57:52,342 INFO sqlalchemy.engine.Engine [generated in 0.00171s] ()
2021-06-04 16:57:52,349 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-06-04 16:57:52,353 INFO sqlalchemy.engine.Engine INSERT INTO v_dw (user_id, user_name, project_id, project_name) VALUES (?, ?, ?, ?)
2021-06-04 16:57:52,356 INFO sqlalchemy.engine.Engine [generated in 0.00254s] (2, 'Kishor', 2, 'B')


OperationalError: (sqlite3.OperationalError) cannot modify v_dw because it is a view
[SQL: INSERT INTO v_dw (user_id, user_name, project_id, project_name) VALUES (?, ?, ?, ?)]
[parameters: (2, 'Kishor', 2, 'B')]
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [50]:
for row in target_session.query(t_v_dw).all():
    print(row)

2021-06-04 16:57:54,551 INFO sqlalchemy.engine.Engine SELECT v_dw.user_id AS v_dw_user_id, v_dw.user_name AS v_dw_user_name, v_dw.project_id AS v_dw_project_id, v_dw.project_name AS v_dw_project_name 
FROM v_dw
2021-06-04 16:57:54,552 INFO sqlalchemy.engine.Engine [generated in 0.00184s] ()
(1, 'Shivam', 1, 'A')
(2, 'Kishor', 2, 'B')


In [51]:
source_engine.dispose()
target_engine.dispose()